<a href="https://colab.research.google.com/github/LCaravaggio/DeepLearning_ITBA/blob/main/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c cifar100-dl-itba-2020q2

Mounted at /content/drive
  0% 0.00/456 [00:00<?, ?B/s]
100% 456/456 [00:00<00:00, 820kB/s]
 75% 20.0M/26.8M [00:00<00:00, 32.4MB/s]
100% 26.8M/26.8M [00:00<00:00, 76.8MB/s]
  0% 0.00/1.30k [00:00<?, ?B/s]
100% 1.30k/1.30k [00:00<00:00, 3.81MB/s]
  0% 0.00/391k [00:00<?, ?B/s]
100% 391k/391k [00:00<00:00, 125MB/s]
 93% 124M/134M [00:01<00:00, 82.7MB/s]
100% 134M/134M [00:01<00:00, 94.4MB/s]
  0% 0.00/391k [00:00<?, ?B/s]
100% 391k/391k [00:00<00:00, 99.0MB/s]


In [5]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [6]:
import numpy as np

x_test=np.load("x_test.npy")
x_train=np.load("x_train.npy")
y_train_coarse=np.load("y_train_coarse.npy")
y_train_fine=np.load("y_train_fine.npy")

#for file in os.listdir():
#    if file.endswith('.npy'):
#      np.load(file)

In [7]:
if not os.path.exists('mlp_helper.py'):
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/mlp_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/fnn_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/MNIST_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/draw_nn.py

--2021-05-21 22:54:05--  https://github.com/lab-ml-itba/MLP-2019/raw/master/mlp_helper.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lab-ml-itba/MLP-2019/master/mlp_helper.py [following]
--2021-05-21 22:54:05--  https://raw.githubusercontent.com/lab-ml-itba/MLP-2019/master/mlp_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13863 (14K) [text/plain]
Saving to: ‘mlp_helper.py’

mlp_helper.py       100%[===================>]  13.54K  --.-KB/s    in 0s      

2021-05-21 22:54:06 (37.9 MB/s) - ‘mlp_helper.py’ saved [13863/13863]

--2021-05-21 22:54:06--  https://github.com/lab-ml-itba/

In [8]:
import os.path
import tensorflow.keras
import numpy as np
from matplotlib import pyplot as plt
from fnn_helper import PlotLosses
from MNIST_helper import plot_numbers, visualize_input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
from fnn_helper import PlotLosses
#from keras_contrib.callbacks import CyclicLR
from matplotlib import pyplot as plt
from mlp_helper import plot_confusion_matrix
#import draw_nn

In [9]:
np.unique(y_train_fine)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [17]:
def get_model(activations=['tanh', 'sigmoid']):
    output_size = 100
    model_plane = Sequential()
    model_plane.add(Flatten(input_shape=x_train.shape[1:]))
    model_plane.add(Dense(10, activation=activations[0],  name='middle1', kernel_initializer='normal'))
    model_plane.add(Dense(5, activation=activations[1],  name='middle2', kernel_initializer='normal'))
    #model_plane.add(LeakyReLU())
    #model_plane.add(LeakyReLU())
    model_plane.add(Dense(output_size, activation='linear',  kernel_initializer='normal', name='salida'))
    return model_plane

In [18]:
model_plane = get_model(['tanh', 'sigmoid'])
model_plane.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 3072)              0         
_________________________________________________________________
middle1 (Dense)              (None, 10)                30730     
_________________________________________________________________
middle2 (Dense)              (None, 5)                 55        
_________________________________________________________________
salida (Dense)               (None, 100)               600       
Total params: 31,385
Trainable params: 31,385
Non-trainable params: 0
_________________________________________________________________


In [35]:
batch_size=10

In [36]:
model_plane = get_model(activations=['tanh', 'linear'])

checkpointer = ModelCheckpoint(filepath='two-layer-2d.mnist.hdf5', verbose=1, save_best_only=True, mode='max', monitor='val_accuracy')
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=None, x_val=x_train, y_val_categorical=y_train_fine)
# clr = CyclicLR(1e-6, 1e-4, 8*int(len(x_train)/batch_size))

In [37]:
optim = RMSprop(lr=1e-4) # 1e-06
model_plane.compile( loss= 'mse', optimizer=optim, metrics=['accuracy'])

In [38]:
model_plane.fit(x_train, 
          y_train_fine ,
          epochs=10, batch_size=batch_size, 
          #verbose=1, 
          validation_data=(x_train, y_train_fine), 
          callbacks=[checkpointer],
         )

Epoch 1/10
5000/5000 [==============================] - 12s 2ms/step - loss: 3105.8091 - accuracy: 0.0102 - val_loss: 2065.9719 - val_accuracy: 0.0100

Epoch 00001: val_accuracy improved from -inf to 0.01000, saving model to two-layer-2d.mnist.hdf5
Epoch 2/10
5000/5000 [==============================] - 11s 2ms/step - loss: 1640.9753 - accuracy: 0.0105 - val_loss: 846.5863 - val_accuracy: 0.0100

Epoch 00002: val_accuracy did not improve from 0.01000
Epoch 3/10
5000/5000 [==============================] - 11s 2ms/step - loss: 839.9980 - accuracy: 0.0100 - val_loss: 833.3661 - val_accuracy: 0.0100

Epoch 00003: val_accuracy did not improve from 0.01000
Epoch 4/10
5000/5000 [==============================] - 11s 2ms/step - loss: 834.8214 - accuracy: 0.0100 - val_loss: 833.3329 - val_accuracy: 0.0100

Epoch 00004: val_accuracy did not improve from 0.01000
Epoch 5/10
5000/5000 [==============================] - 11s 2ms/step - loss: 834.0934 - accuracy: 0.0095 - val_loss: 833.4073 - val_acc

In [36]:
model_plane.evaluate(x_train, y_train_fine)

1563/1563 [==============================] - 2s 1ms/step - loss: 0.5149 - accuracy: 0.0100


[0.5149050951004028, 0.009999999776482582]